# Build a Kriging surrogate on some dummy prop data

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dummy_prop_example
from smt.sampling_methods import FullFactorial

%load_ext autoreload
%autoreload 2

**First let us consider a 'low fidelity' dataset. We could build a full factorial of data if this is cheap, so lets pretend to do that.**

We also may choose to remove some cases, since we can't always successfully manage to build a full factorial set of structured data. 

Let us set up the sampling limits:

*Note:* that we use the kwarg `clip=True` in order for the number of points to give a full grid.

In [21]:
# setup each limit
discangle_limits = [-90.0, 90.0]
propspeed_limits = [400.0, 1200.0]
airspeed_limits = [0.0, 60.0]
# and we group them into a list
xlimits = np.array([discangle_limits, airspeed_limits, propspeed_limits])
# and make the full factorial sampling
lf_sampling = FullFactorial(xlimits=xlimits, clip=True)
number_of_lf_samples = 200
lf_independent = lf_sampling(number_of_lf_samples)
print(f"Actual number of samples generated is: {lf_independent[:, 0].shape[0]}")

Actual number of samples generated is: 216


**now we can plot the inputs, just to see what we have achieved**

In [83]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=lf_independent[:, 0],
            y=lf_independent[:, 1],
            z=lf_independent[:, 2],
            mode="markers",
            marker=dict(color="LightSkyBlue", size=4),
        )
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title="disc angle [deg]",
        yaxis_title="airspeed [m/s]",
        zaxis_title="prop speed [RPM]",
    ),
)

fig.show()


**It is now time to build a Pandas DataFrame, containg all of this data**

We don't really need to do this to build a Kirging surrogate, however we are doing this as it is common that the data won't just come out as a tidy NumPy array, and instead will be loads from some .csv file or similar.

*Note:* that the arguments after the sampling points are used to caluclate the dependent variable and are unique to the propeller example we are running.

In [64]:
lf_data_df: pd.DataFrame = dummy_prop_example.lf_data(
    lf_independent[:, 0],
    lf_independent[:, 1],
    lf_independent[:, 2],
    1000.0,
    4000.0,
)

display(lf_data_df)

,airspeed,discangle,propspeed,load
0,0.0,-90.0,400.0,640.0
1,0.0,-90.0,560.0,1254.4
2,0.0,-90.0,720.0,2073.6
3,0.0,-90.0,880.0,3097.6
4,0.0,-90.0,1040.0,4326.4
...,...,...,...,...
211,60.0,90.0,560.0,12774.4
212,60.0,90.0,720.0,13593.6
213,60.0,90.0,880.0,14617.6
214,60.0,90.0,1040.0,15846.4


**At this stage, it is probably worthwhile to see what the data looks like**

*Note:* that this data and the trends are made up, the trends are also made up

In [81]:
rpms = np.unique(lf_data_df.propspeed.to_numpy())
rpm = rpms[-1]
plot_trend_df = lf_data_df[lf_data_df.propspeed == rpm]

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=plot_trend_df.discangle,
            y=plot_trend_df.airspeed,
            z=plot_trend_df.load,
            mode="markers",
            marker=dict(
                color="LightSkyBlue", size=4),
        )
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title="disc angle [deg]",
        yaxis_title="airspeed [m/s]",
        zaxis_title="thrust [N]",
    ),
    title=f"Prop speed is {rpm} RPM",
)
